#### Ridge Regression on Polynomial Features

Now we will implement this polynomial features regression with cross validation and ridge regression. It is important to remember that we have to standardize our variables before doing so.

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
# read in data
df = pd.read_csv('../data/Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


Split the data twice so that we have a holdout set. If we do cross validation (or just a regular train test split), after we update the parameters we will have seen data from the test set and optimized based on that data which is data leakage. Cross-validation with a holdout set is basically a way to test on our data prior to using the actual testing data.

In [33]:
# set up X and y then do splits
X = df.drop('sales', axis = 1)
y = df['sales']

# train test split for now
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# holdout set - dont forget to run preprocessing on this later
X_train_val, X_test_holdout, y_train_val, y_test_holdout = train_test_split(X, y, test_size=0.2, random_state=42)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

This would normally be done in a pipeline but we are just going through the steps for clarity this time.

In [34]:
# polynomial set up
poly = PolynomialFeatures(degree=3, include_bias=False)

# fit transform training, transform testing
X_train_poly = poly.fit_transform(X_train)

# transform testing
X_test_poly = poly.transform(X_test)

In [35]:
# standard scaler
scaler = StandardScaler()

# fit transform training, transform testing
X_train_scaled = scaler.fit_transform(X_train_poly)

X_test_scaled = scaler.transform(X_test_poly)

In [36]:
# ridge regression
from sklearn.linear_model import Ridge

In [37]:
# create model
ridge = Ridge(alpha=10)

# fit
ridge.fit(X_train_scaled, y_train)

# predict
y_pred = ridge.predict(X_test_scaled)

In [38]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(y_test, y_pred)
rmse

0.7423203506591112

In [39]:
# using cross validation
from sklearn.model_selection import KFold, cross_val_score

# set up our folds
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# cross val scores
scores = cross_val_score(ridge, X_train_scaled, y_train, scoring='neg_root_mean_squared_error', cv = cv)

In [40]:
scores

array([-0.93288886, -0.96202715, -1.64368222, -0.9003756 , -0.73291189])

In [41]:
# get mean of scores
np.mean(scores)

np.float64(-1.0343771451925177)

In [43]:
# test on holdout set
X_test_holdout_poly = poly.transform(X_test_holdout)
X_test_holdout_scaled = scaler.transform(X_test_holdout_poly)

In [44]:
holdout_pred = ridge.predict(X_test_holdout_scaled)
rmse_holdout = root_mean_squared_error(y_test_holdout, holdout_pred)

In [45]:
rmse_holdout

0.8933124148305153